<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Método Actor-Crítico</center></span>



<figure>
<center>
<img src="../Imagenes/Atlantis_seen_over_the_Mediterranean_Sea.jpeg" width="800" height="400" align="center"/>
<figcaption>Templo de Confucio</figcaption>
</center>
</figure>

<a href="https://commons.wikimedia.org/wiki/File:Atlantis_seen_over_the_Mediterranean_Sea,_near_the_Algerian_coast.jpeg">NASA</a>, Public domain, via Wikimedia Commons

##   <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

## <span style="color:blue">Referencias</span>

1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2021](https://github.com/AprendizajeProfundo/Diplomado)
1. [Maxim Lapan, Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)
1. [Adaptado de Rowel Atienza, Advance Deep Learning with Tensorflow 2 and Keras,Pack,2020](https://www.amazon.com/-/es/Rowel-Atienza-ebook/dp/B0851D5YQQ).
1. [Sutton, R. S., & Barto, A. G. (2018).Reinforcement learning: An introductio, MIT Press, 2018](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf)
1. [Ejecutar en Colab](https://colab.research.google.com/drive/1ExE__T9e2dMDKbxrJfgp8jP0So8umC-A#sandboxMode=true&scrollTo=2XelFhSJGWGX)
1. [Human-level control through deep reinforcement
learning](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) 

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Refuerzo con línea base](#Refuerzo-con-línea-base)
* [Método Actor-crítico ](#Método-Actor-crítico )

## <span style="color:blue">Introducción</span>

En esta lección exploramos el `método actor-crítico` que es una variante de los métodos básicos de [gradiente de política](ar_Gradientes_Politica.ipynb) estudiados antes. Este método mejora de forma casi mágica la estabilidad y velocidad de convergencia.

Recordemos primero el método `Reinforce` con línea base introducido en la lección de gradiente de política.

## <span style="color:blue">Refuerzo con línea base </span>


Ya mencionamos antes que nuestra aproximación del gradiente de la función objetivo es dada por $\nabla J \approx \mathbb{E}[Q(s,a)\nabla \pi(s|a)]$ es proporcional a la recompensa descontada desde un estado dado. Esta recompensa depende por supuesto del ambiente e introduce una fuente importante de variación. Una solución a este inconveniente es substraer de la recompensa una constante. Posibles soluciones son restar de la recompensa algún valor como por ejemplo:
 
+ La media de las recompensas descontadas.
+ La media móvil de las recompensas descontadas.
+ El valor del estado V(s).

La última solución es introducida en esta lección actor-crítico. Recuerde que el valor del estado $V(s)$ es definido como el retorno esperado partiendo del estado $s$.

Por otro lado para recortar los episodios y no hacer cálculos innecesarios se puede determinar a partir de que momento el factor de descuento es tan pequeño que más pasos en el episodio no aportan mucho al cálculo de la recompensa descontada. Por ejemplo, $0.9^{50}= 0.005$. Entonces si el factor de descuento es $\gamma=0.9$, quizás sea suficiente parar después de 50 pasos en un episodio.


La introducción de un constante que reste a la recompensa descontada se basa en el siguiente hecho.

Puede verificarse que

$$
\mathbb{E}_{\theta}\left[\left( \nabla \log \pi_{\theta}(a_t|s_t) \right) \right]=0.
$$

En efecto, bajo el supuesto que la integral y el gradiente pueden intercambiarse, se tiene que

$$
\mathbb{E}_{\theta}\left[\left( \nabla \log \pi_{\theta}(a|s) \right) \right] =\int \pi_{\theta}(a|s)\nabla \log \pi_{\theta}(a|s)d\tau =  \int \nabla \pi_{\theta}(a|s)d\tau= \nabla\int  \pi_{\theta}(a|s)d\tau = \nabla  1 = 0.
$$

La verificación meticulosa, se  deja como ejercicio al lector interesado. Sea $b$ una variable que no depende de la trayectoria $\tau$.  Esta variable se denominará `línea base`. Debido al resultado anterior se tiene que

$$
\nabla J(\theta) = \nabla \mathbb{E}_{\pi_{\theta}}[r(\tau)] = \mathbb{E}_{\pi_{\theta}}\left[  \sum_{t=1}^{T}(G_t-b)\nabla\log \pi_{\theta}(a_t|s_t) \right].
$$

Usando $b$ en teoría y en la práctica la varianza puede ser reducida, manteniendo el gradiente de la política insesgado. Un buen valor que puede usarse como línea base el valor del estado actual.

## <span style="color:blue">Método actor-crítico </span>

### <span style="color:#4CC9F0"> Reescritura de la recompensa total</span>

Recordemos que en la lección de extensiones de DQN, en particular en la extensión Dueling DQN indicamos que  la recompensa total puede escribirse en la forma

$$
Q(s,a) = V(s) + A(s,a)
$$

en donde $V(s)$ es el valor del estado $s$ y $A(a,s)$ es la ventaja de seleccionar la acción $a$.

Entonces la pregunta que surge es: ¿Y si usamos como línea base $V(s)$?. Esto parece una idea interesante porque ahora el gradiente tendría un peso asociado directamente a la ventaja de selecciona la acción $a$.

En realidad, esta es una gran idea. Pero tenemos un problema. No disponemos del valor $V(s)$.

Para resolverlo introducimos una segunda red que estime el valor del estado.

Entonces, tendremos dos redes, que pueden ser implementadas en una sola unidad neuronal. Esta redes son

+ Red de la política (actor)
+ Red del valor (crítico)



<figure>
<img src="../Imagenes/actor-critic_01.png"  width="400" height="400" align="center"/> 
</figure>

Fuente: Maxi Lapan

La idea central detrás de este método es la siguiente. La red política (actor) dice que hacer. A su vez la red valor (crítico) dice que tan buenas fueron las acciones.

En la práctica las dos redes se traslapan principalmente por razones de eficiencia y convergencia.  Ambas redes se implementan como dos cabeza de una red con un cuerpo común, tal como se muestra en la siguiente imagen.



<figure>
<img src="../Imagenes/actor-critico_2.png"  width="400" height="400" align="center"/> 
</figure>

Fuente: Maxi Lapan

### <span style="color:#4CC9F0"> Algortimo actor-crítico (A2C)</span>

1. Inicializa los parámetros de la red con valores aleatorios.
2. Corre N pasos en el ambiente usando la actual política $\pi_{\theta}$, y almacenando $(s_t, a_t, r_t)$.
3. $R=0$ si el final del episodio es alcanzado. Si no $R = V_{\theta}(s)$.
4. Para $i=t-1, \ldots, t_{start}$ (los pasos son procesados hacia atrás)
    + $R = r_i + \gamma R$
    + Acumula gradientes de política: $\partial \theta_{\pi} = \partial \theta_{\pi} + \nabla_{\theta} \log \pi_{\theta}(a_i|s_i)(R-V_{\theta}(s_i))$
    + Acumula gradientes de valor: $\partial \theta_{\nu} = \partial \theta_{\nu} + \tfrac{\partial(R- V_{\theta}(s_i))}{\partial \theta_{\nu}}$
5. Actualiza los parámetros de la red usando los gradientes acumulados, moviendo en la dirección de los gradientes de la política, $\partial \theta_{\pi}$ y en la dirección opuesta de los gradientes de valor $\partial \theta_{\nu}$.
6. Repite desde 2 hasta alcanzar convergencia.

### <span style="color:#4CC9F0"> Nota de implementación</span> 

Por lo general la función de pérdida incluye la pérdida de la política, la pérdida de valor y una perdida entropía, la cual introduce el efecto de empujar al agente lejos de tener certidumbre acerca de sus acciones. La entropía introducida es $$
\mathfrak{L}_H = \beta\sum_t \pi_{\theta}(s_i)\log \pi_{\theta}(s_i)
$$
Aquí $\beta$ es un hiperparámetro en el problema. Por ejemplo tomaremos $\beta= 0.01$ en nuestro ejemplo. 

Si $\mathfrak{L}_{\pi}$ denota la función de pérdida asociada a la política y $\mathfrak{L}_V$ la pérdida asociada al valor del estado, la pérdida total de la técnica actor crítico (A2C) es dada por 

$$
\mathfrak{L}_{A2C} = \mathfrak{L}_{\pi} + \mathfrak{L}_V + \mathfrak{L}_H
$$

Observe que $\mathfrak{L}_H$ actúa como un regularizador para la función de pérdida.

Finalmente, para mejorar la estabilidad es usual trabajar con varios ambientes, los cuales entregan observaciones de manera concurrente.

## <span style="color:blue">Ejemplo: A2C en Pong </span>

Esta es una implementación del entrenamiento de actor-crítico para el juego Pong. Usaremos la librería [Ptan](https://github.com/Shmuma/ptan) del profesor Max Lapan.

### <span style="color:#4CC9F0">  Importa módulos</span>

In [ ]:
#!/usr/bin/env python3
import gym
import ptan
import numpy as np
import argparse
import collections
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn.functional as F
import torch.nn.utils as nn_utils
import torch.optim as optim

from lib import common

### <span style="color:#4CC9F0">  Hiperparámetros</span>

In [ ]:
GAMMA = 0.99
LEARNING_RATE = 0.0001
ENTROPY_BETA = 0.01
BATCH_SIZE = 128

REWARD_STEPS = 10
BASELINE_STEPS = 1000000
GRAD_L2_CLIP = 0.1

ENV_COUNT = 32

### <span style="color:#4CC9F0"> Clase Atari2C</span>

El primer objeto importante es la implementación de la red neuronal que se utilizará. La siguiente celda muestra la implementación completa para el ejemplo que hemos considerado.

Como se dijo antes, se tiene un cuerpo común y dos cabezas de red, una para la política (policy) y la otra para el valor (value). 

Similarmente a como hicimos en el método [Reinforcement](ar_Gradientes_Politica.ipynb), la red tiene dos cabezas que se desprenden del cuerpo común. Sin embargo, en este caso, el método *forward()* retorna el tensor de los logits de la política y el tensor del valor en forma separada. Es decir, la salida de la red es $[q_{\theta}(s), V_{\theta}(s)]$. La red entrega una pre-probabilidad (logits). Para obtener $\pi_{\theta}(s)$, se calcula 

$$
\pi_{\theta}(s) = \text{softmax} (q_{\theta}(s))
$$.

In [ ]:
class AtariA2C(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(AtariA2C, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
            )
        conv_out_size = self._get_conv_out(input_shape)
        
        self.policy = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
            )
        
        self.value = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
            
    def forward(self, x):
        fx = x.float() / 256
        conv_out = self.conv(fx).view(fx.size()[0], -1)
        return self.policy(conv_out), self.value(conv_out)
    

### <span style="color:#4CC9F0"> Función unpack_batch()</span>

Ahora tenemos una función grande e importante, que toma la
lote de transiciones de entorno y devuelve tres tensores: el lote de estados,
lote de acciones realizadas y lote de valores Q calculados utilizando la fórmula

$$
Q(s,a) = \sum_{i=0}^{N-1} \gamma^i r_i + \gamma^NV(s_N)
$$

Este valor $Q$ se utilizará en dos lugares: 
 * para calcular pérdida del **error cuadrático medio (MSE)** para mejorar la aproximación del valor en el mismo manera como DQN, y 
 * para calcular la ventaja de la acción.

In [ ]:
def unpack_batch(batch, net, device='cpu'):
    states = []
    actions = []
    rewards = []
    not_done_idx = []
    last_states = []
    for idx, exp in enumerate(batch):
        states.append(np.array(exp.state, copy=False))
        actions.append(int(exp.action))
        rewards.append(exp.reward)
        if exp.last_state is not None:
            not_done_idx.append(idx)
            last_states.append(
                np.array(exp.last_state, copy=False))

    states_v = torch.FloatTensor(
        np.array(states, copy=False)).to(device)
    actions_t = torch.LongTensor(actions).to(device)

    rewards_np = np.array(rewards, dtype=np.float32)

    if not_done_idx:
        last_states_v = torch.FloatTensor(
            np.array(last_states, copy=False)).to(device)
        last_vals_v = net(last_states_v)[1]
        last_vals_np = last_vals_v.data.cpu().numpy()[:, 0]
        last_vals_np *= GAMMA ** REWARD_STEPS
        rewards_np[not_done_idx] += last_vals_np

    ref_vals_v = torch.FloatTensor(rewards_np).to(device)
    return states_v, actions_t, ref_vals_v

### <span style="color:#4CC9F0"> Entrenamiento</span>

Finalmente presentamos el código de entrenamiento.   

**Para hacer (TODO):** Transformar este código en un Trainer como hemos hecho en otras lecciones.  

In [ ]:
if __name__ == "__main__":
    # preparativos
    parser = argparse.ArgumentParser()
    parser.add_argument("--cuda", default=False,
                        action="store_true", help="Enable cuda")
    parser.add_argument("-n", "--name", required=True,
                        help="Name of the run")
    args = parser.parse_args()
    device = torch.device("cuda" if args.cuda else "cpu")
    
    # instancia los ambientes que trabajaran en paralelo
    make_env = lambda: ptan.common.wrappers.wrap_dqn(
            gym.make("PongNoFrameskip-v4"))
    envs = [make_env() for _ in range(NUM_ENVS)]
    writer = SummaryWriter(comment="-pong-a2c_" + args.name)

    # instancia la red neuronal
    net = AtariA2C(envs[0].observation_space.shape,
                    envs[0].action_space.n).to(device)
    print(net)
    
    # instancia el agente
    agent = ptan.agent.PolicyAgent(
            lambda x: net(x)[0], apply_softmax=True, device=device)
    
    # instancia el buffer de experiencia esta equipado para 
    # trabajar en paralelo  con varios ambientes
    exp_source = ptan.experience.ExperienceSourceFirstLast(
            envs, agent, gamma=GAMMA, steps_count=REWARD_STEPS)
    # instancia el optimizador para los parámetros de la red neuronal
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE,
                eps=1e-3)
    
    # crea la lista para recibir los lotes de datos del buffer de experiencia
    batch = []
    
    # crea un contexto para ejecutar el ciclo de entrenamiento para controlar cuando se alcanza 
    # la frontera de recompensa  (18 en este caso)
    with common.RewardTracker(writer, stop_reward=18) as tracker:
        # crea un contexto para enviar la información a Tensorboard
        with ptan.common.utils.TBMeanTracker(writer,
            batch_size=10) as tb_tracker:
            # empieza a extraer los datos del buffer de experiencia y
            # a colocarlos en la lista batch
            for step_idx, exp in enumerate(exp_source):
                batch.append(exp)
                new_rewards = exp_source.pop_total_rewards()
                # termina si alcnza el umbral de recompensa
                if new_rewards:
                    if tracker.reward(new_rewards[0], step_idx):
                        break
                # verifica si ya tiene lel tamaño del batch definido
                if len(batch) < BATCH_SIZE:
                    continue
                # usa la función unpack_batch para extraer estados, acciones y q-valores
                # del lote (batch) de datos
                states_v, actions_t, vals_ref_v = \
                        unpack_batch(batch, net, device=device)
                # limpia la lista del lot3e de datos para el siguiente paso
                batch.clear()
                
                # paso de optimización
                
                # gradientes en cero
                optimizer.zero_grad()
                
                # calcula logits de la política y valores de estados con la red neuronal
                logits_v, value_v = net(states_v)
                
                # calcula la función de pérdida (mse) para el valor del estado V(s): L_V
                loss_value_v = F.mse_loss(
                                value_v.squeeze(-1), vals_ref_v)
                    
                # calcula el logaritmo de la política a partir de los logits, usando log (softmax)
                log_prob_v = F.log_softmax(logits_v, dim=1)
                
                # Calcula la ventaja V(a,s) normalizando con el valor del estado
                # para disminuir la varianza en el muestreo para calcular la integral
                adv_v = vals_ref_v - value_v.detach()
                
                # calcula el logaritmo de la probabilidad para toda slas acciones del lote de datos
                log_p_a = log_prob_v[range(BATCH_SIZE), actions_t]
                # multiplica por la ventaja
                log_prob_actions_v = adv_v * log_p_a
                # aproxima la integral (MCMC) para obtener la pérdida de la política: L_{\pi}
                loss_policy_v = -log_prob_actions_v.mean()    
                
                # Calcula la entropia para regularizar la funcipon de peŕdida: L_{\beta}
                prob_v = F.softmax(logits_v, dim=1)
                ent = (prob_v * log_prob_v).sum(dim=1).mean()
                entropy_loss_v = ENTROPY_BETA * ent
                
                #  Calcula los gradientes de la política
                loss_policy_v.backward(retain_graph=True)
                grads = np.concatenate([
                            p.grad.data.cpu().numpy().flatten()
                            for p in net.parameters()  if p.grad is not None
                            ])
                
                # calcula los gradientes del valor de estado (adiciona la regularización L_{\beta})
                loss_v = entropy_loss_v + loss_value_v
                loss_v.backward()
                # recorta los gradientes de ser necesario
                nn_utils.clip_grad_norm_(net.parameters(), CLIP_GRAD)
                
                # paso por el optimizador
                optimizer.step()
                
                # acumula la pérdida total para colocar en Tensorboard
                loss_v += loss_policy_v
                
                # para seguimiento en tensorboard
                tb_tracker.track("advantage", adv_v, step_idx)
                tb_tracker.track("values", value_v, step_idx)
                tb_tracker.track("batch_rewards", vals_ref_v, step_idx)
                tb_tracker.track("loss_entropy", entropy_loss_v, step_idx)
                tb_tracker.track("loss_policy", loss_policy_v, step_idx)
                tb_tracker.track("loss_value", loss_value_v, step_idx)
                tb_tracker.track("loss_total", loss_v, step_idx)
                g_l2 = np.sqrt(np.mean(np.square(grads)))
                tb_tracker.track("grad_l2", g_l2, step_idx)
                g_max = np.max(np.abs(grads))
                tb_tracker.track("grad_max", g_max, step_idx)
                g_var = np.var(grads)
                tb_tracker.track("grad_var", g_var, step_idx)